In [ ]:
### Script adapted from "ToF ToF covariance slices and yield.ipynb" (by Tiffany Walmsley, Burt Group) ###

# 1. Imports and functions

import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.signal import savgol_filter
from numba import njit
import h5py

u = 1.66e-27
e = 1.6e-19

def create_vel_image(df, nbins, bin_size, Ti = 0, Tf = 0):
    
    if (Ti & Tf):
        df = df[(df['ToF'] >= Ti) & (df['ToF'] <= Tf)]
    xedges = np.arange(-nbins/2, nbins/2)*bin_size
    yedges = np.arange(-nbins/2, nbins/2)*bin_size
    H, xedges, yedges = np.histogram2d(df['vx'], df['vy'], bins = (xedges, yedges))
    
    return(H, xedges, yedges)

def create_pos_image(df, nbins, bin_size, Ti = 0, Tf = 0):
    
    if (Ti & Tf):
        df = df[(df['ToF'] >= Ti) & (df['ToF'] <= Tf)]
    xedges = np.arange(-nbins/2, nbins/2 + 1)*bin_size
    yedges = np.arange(-nbins/2, nbins/2 + 1)*bin_size
    H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = (xedges, yedges))
    
    return(H, xedges, yedges)
    
def create_ToF(df, n_bins, min_t = 0, max_t = 0):
    
    if (max_t):
        intensity, bin_edges = np.histogram(df.ToF, n_bins, range = (min_t, max_t))
    else:
        intensity, bin_edges = np.histogram(df.ToF, n_bins)
    mid_values = (bin_edges[:-1] + (bin_edges[1] - bin_edges[0])/2)
    
    return(mid_values, intensity)

def create_p_dist(p, n_bins, min_p = 0, max_p = 0):
    
    if (max_p):
        intensity,bin_edges = np.histogram(p, n_bins, range = (min_p, max_p))
    else:
        intensity,bin_edges = np.histogram(p, n_bins)
    mid_values = (bin_edges[:-1] + (bin_edges[1] - bin_edges[0])/2)
    
    return(mid_values, intensity)

def beautify(ax, param_dict):
    
    ax.tick_params(labelsize = param_dict['tick_label_size'],
                   length = param_dict['tick_length'],
                   width = param_dict['tick_width'])
    
    ax.xaxis.get_label().set_fontsize(param_dict['axis_label_size'])
    ax.yaxis.get_label().set_fontsize(param_dict['axis_label_size'])
    
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(param_dict['spine_width'])

# param_dict = {'spine_width':1,
#               'line_width':2,
#               'tick_length':3,
#               'tick_width':1,
#               'tick_label_size':12,
#               'axis_label_size':12}

# plt.rcParams["font.family"] = "serif"
# plt.rcParams['mathtext.fontset'] = 'stix'

@njit
def reindex_shots(shot_list):
    
    for i, shot in enumerate(shot_list):
        if shot == shot_list[i+1]: continue
        if shot == shot_list[i+1]-1: continue 
        if shot != shot_list[i+1]-1:
            shot_list[i+1:] -= (shot_list[i+1] - shot)
            
    return(shot_list)


def p_calibration(m_list, q_list, ToF_list, ToF_range_list):
    
    SI_to_au = 1.992e-24
    ion_df_list = []

    for ToF, m, q, (Ti,Tf) in zip(ToF_list, m_list, q_list, ToF_range_list):

        m *= u

        # Collect data
        ion_df = (event_df[(event_df['ToF'] >= Ti) & (event_df['ToF'] <= Tf)]).copy()
        ion_df['ToF_SI_cal'] = (ion_df['ToF'] - ion_T0)*1e-9

        ion_df['px_SI'] = ion_df['x']*0.826*m/ion_df['ToF_SI_cal']
        ion_df['py_SI'] = ion_df['y']*0.826*m/ion_df['ToF_SI_cal']
        ion_df['pz_SI'] = -1.9e-15*2*q*((ion_df['ToF'] - ToF)*1e-9)
        ion_df['pmag_SI'] = np.sqrt(ion_df['pz_SI']**2 + ion_df['py_SI']**2 + ion_df['px_SI']**2)

        for column_pair in [('px_AU', 'px_SI'), ('py_AU', 'py_SI'), ('pz_AU', 'pz_SI'), ('pmag_AU', 'pmag_SI')]:
            ion_df[column_pair[0]] = ion_df[column_pair[1]]/SI_to_au

        ion_df['pr_AU'] = np.sqrt(ion_df['px_AU']**2 + ion_df['py_AU']**2)

        ion_df_list.append(ion_df)

    return(ion_df_list)

In [ ]:
# 2. Loading in data 

import os

pd.set_option('display.max_rows', 4096) # Controls number of lines displayed (PImMS maximum = 4096 timebins)

#directory=os.chdir(r"/home/merrickj/Documents/indene_filtered_data_for_covariance")
directory=os.chdir(r"/home/merrickj/Documents/fluorene_filtered_data_for_covariance")
#directory=os.chdir(r"/home/merrickj/Documents/CPP_filtered_data_for_covariance")

#file_list = ['038','039','040','041','042','043','044'] # run number; adapt to supply an array of run numbers whose data can be aggregated over
#file_list = ['indene_full_data_centroided_corrected']
file_list = ['fluorene_full_data_centroided_corrected']
#file_list = ['CPP_full_data_centroided_corrected']
df_list = []

for f in zip(file_list):
    print(f[0])
    print('Opening file: '+f[0])
    data = np.load(f"{f[0]}.npy", mmap_mode='r')
    #df = pd.DataFrame(data, columns = ['Energy','x','y','ToF','tag_ID','delay']) # 2021 data
    #df = pd.DataFrame(data, columns = ['x','y','ToF','tId','delay','tag_ID','Energy']) # 2023 on the fly
    df = pd.DataFrame(data, columns =  ['Energy','tag_ID','x','y','ToF','size','spread','delay','m_z']) # indene centroided data 2023
    df['tag_ID'] = df['tag_ID'] .astype(int)
    print('# shots = '+str(len(np.unique(df['tag_ID']))))
    print('# events = '+str(len(df)))
    df_list.append(df)
    print(df)

In [ ]:
# 3. ToF-ToF covariance function

# Imports and functions

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import time
import os
from numba import njit
import h5py
from scipy.ndimage import gaussian_filter

@njit
def reindex_shots(shot_list):
    
    for i, shot in enumerate(shot_list):
        if shot == shot_list[i+1]: continue
        if shot == shot_list[i+1]-1: continue 
        if shot != shot_list[i+1]-1:
            shot_list[i+1:] -= (shot_list[i+1] - shot)
            
    return(shot_list)

@njit
def ToF_ToF_coincidence(Sij, frame_array_A, shot_list, min_ToF):
    
    no_A = frame_array_A.shape[0]
    nshotsA, nendA = 0, 0
    
    for shot in shot_list:
        
        A_in_shot = False
        
        for test in range(nshotsA, no_A):
            if (frame_array_A[test, 1] == shot):
                nshotsA = test
                A_in_shot = True
                break
                
        if A_in_shot:
            for test in range(nshotsA, no_A):
                if frame_array_A[test, 1] > shot:
                    nendA = test
                    break
                
        if (nendA - nshotsA) > 1:
            
            for i in range(nshotsA, nendA):
                A_index = frame_array_A[i, 0] - min_ToF -1
                
                for j in range(nshotsA, nendA):
                        B_index = frame_array_A[j, 0] - min_ToF - 1
                        
                        Sij[A_index, B_index] += 1

    return(Sij)

def ToF_ToF_covariance(event_df, min_ToF, max_ToF):

    df_filt = (event_df[(event_df['ToF'] >= min_ToF) & (event_df['ToF'] <= max_ToF)]).copy()
    df_filt['ToF'] = (df_filt['ToF'] + 0.5).astype('int64')
    event_array = np.array(df_filt[['ToF','tag_ID']])

    shot_list = np.unique(event_array[:, 1])
    n_shots = len(shot_list)

    # Sij
    
    ToF_range = int(max_ToF - min_ToF)
    Sij = np.zeros((ToF_range, ToF_range))
    Sij = ToF_ToF_coincidence(Sij, event_array, shot_list, min_ToF)
    Sij /= n_shots
    
    #SiSj
    
    unique, counts = np.unique(event_array[:, 0], return_counts = True)
    ToF_array = np.zeros((ToF_range))
    for count, unique in zip(counts, unique):
        ToF_array[unique - min_ToF - 1] += count
    ToF_array /= n_shots
    SiSj = np.outer(ToF_array, ToF_array)
    
    return(Sij - SiSj, Sij, SiSj)



In [ ]:
# 4. calculate tof tof cov; contingent covariance adaptation coded by JHM

covar_list = []

covar_contingent_store = []

number_of_bins = 5 # for contingent covariance (in reality, number of bins is one minus from this value) (JHM)

for df in zip(df_list): # looping over each run number to sort energy bin stuff out
        
    df = df[0].sort_values(by=['Energy']) # load dataframe in ascending energy value
    df_length = df.shape[0] # number of rows in total dataframe

    min_energy = df['Energy'].min()
    max_energy = df['Energy'].max()

    FEL_energy_bins = []

    index_range = np.arange(0, df_length, df_length/number_of_bins)
    index_range_int = []

    for a in range(len(index_range)): # need to make all values integers
        index_range_int.append(int(index_range[a]))

    print(f"Integer index range: {index_range_int}")

    for index in index_range_int:
        indexed_energy = df['Energy'].iloc[index]
        FEL_energy_bins.append(indexed_energy)

    print(f"FEL energy bin values: {FEL_energy_bins}")
    
for i, df in enumerate(zip(df_list)):
    df = df[0]

    print('Opening file: '+str(i))
    count = 1
    
    for i in range(len(FEL_energy_bins)- 1):
        
        FEL_lower = FEL_energy_bins[i]
        FEL_higher = FEL_energy_bins[i+1]
        
        FEL_filtered_df = df[(df['Energy'] >= FEL_lower) & (df['Energy'] <= FEL_higher)]
        #print(FEL_filtered_df)
        
        #min_ToF, max_ToF = 1000, 1350 for triphenylene
        
        #min_ToF, max_ToF = 1550, 1800 # for indene; 2023 beamtime data
        min_ToF, max_ToF = 1500, 1900 # for fluorene; 2023 beamtime data
        
        ToF_range = max_ToF - min_ToF
        covar_sum_2 = np.zeros((ToF_range, ToF_range))
        sij_sum_2 = np.zeros((ToF_range, ToF_range))
        sisj_sum_2 = np.zeros((ToF_range, ToF_range))
        start_time = time.time()

        covar,Sij,SiSj = ToF_ToF_covariance(df, min_ToF, max_ToF)
        print('Calculated covariance for bin %s at %s seconds'  % (count, round((time.time() - start_time), 1)))
        covar_sum_2 += covar
        sij_sum_2 += Sij
        sisj_sum_2 += SiSj

        covar_contingent_store.append(covar_sum_2)
        count += 1
    
    covar_list.append(sum(covar_contingent_store))

covar = sum(covar_list)
print('Done!')

    

In [ ]:
# 5. plotting tof tof covariance

import matplotlib

fig,ax = plt.subplots(figsize=(10,10))
shw = ax.imshow(covar, cmap='inferno', origin='lower',vmin=0, vmax=1.5*np.max(covar)/50)
ax.set_xlim(0,(max_ToF - min_ToF))
ax.set_ylim(0,(max_ToF - min_ToF))
plt.show()

In [ ]:
# 6. TOF-TOF covariance plot but with TOF spectra on axes (JHM)

from mpl_toolkits.axes_grid1 import make_axes_locatable

# TOF spectrum plotting
tof_appended = []
for df,file in zip(df_list,file_list):
    print('Opening file: '+str(file[0]))
    #df = df[(df['ToF']>1500) & (df['ToF']<1900)]
    tof = df.groupby(['ToF']).size().reset_index(name='intensity')
    tof_times_list = tof['ToF'].to_list()
    tof_appended.append(tof)

list_of_tof_intensities = []

for tof_frame in zip(tof_appended):
    tof_frame = tof_frame[0]
    tof_intensity = tof_frame['intensity'].to_list()
    list_of_tof_intensities.append(tof_intensity)

total_summed_intensities = [sum(i) for i in zip(*list_of_tof_intensities)]

fig, ax = plt.subplots(figsize=(15, 6))
plt.plot(tof_times_list,total_summed_intensities)

ax.set_xlim(1550,1800)
ax.grid(which='major')
ax.grid(which='minor')
ax.minorticks_on()
plt.xlabel('Time-of-flight / arb. units', fontsize=16)
plt.ylabel('Intensity / arb. units', fontsize=16)
plt.title('Total ToF spectrum')
#fig.savefig('triphenylene_total_tof.png')

tof_times_list = [x-min_ToF for x in tof_times_list]

# Plotting 
fig = plt.figure(figsize=(12, 12))
gs = fig.add_gridspec(2, 2,  width_ratios=(3, 1), height_ratios=(1, 3), left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0, hspace=0)
ax = fig.add_subplot(gs[1, 0])

ax_x = fig.add_subplot(gs[0, 0], sharex=ax) # TOF plot (top row)
ax_y = fig.add_subplot(gs[1, 1], sharey=ax) # TOF plot (column)

ax_x.plot(tof_times_list,total_summed_intensities, color = 'purple')
ax_y.plot(total_summed_intensities, tof_times_list, color = 'purple')
    
#ax.set_xlim(0,300)
ax_x.grid(which='major')
ax_x.grid(which='minor', linewidth=0.3)
ax_x.minorticks_on()
ax_y.grid(which='major')
ax_y.grid(which='minor', linewidth=0.3)
ax_y.minorticks_on()
ax_x.tick_params(axis="x",direction="out", pad=5)
ax_y.tick_params(axis="y",direction="out", pad=5)
ax.set_xlabel('Time-of-flight / a.u. ', fontsize=14,labelpad=15)
ax.set_ylabel('Time-of-flight / a.u. ', fontsize=14,labelpad=15)
ax_x.set_ylabel('Intensity / a.u. ', fontsize=14,labelpad=15)
ax_y.set_xlabel('Intensity / a.u. ', fontsize=14,labelpad=15)      
plt.setp(ax_x.get_xticklabels(), visible=False)
plt.setp(ax_y.get_yticklabels(), visible=False)
        
ax.grid(which='major')
ax.grid(which='minor', linewidth=0.3)
ax.minorticks_on()
        
picture = ax.imshow(covar, interpolation='nearest', cmap='inferno', vmin=0, vmax=np.max(covar)/50) # edit vmax as necessary

# colorbar formatting
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
# fig.colorbar(picture, cax=cbar_ax)
# cbar_ax.set_ylabel('Covariance / a.u.', fontsize = 14, rotation = 270, labelpad=25)

divider = make_axes_locatable(ax_y)
cax = divider.append_axes("right", size="10%", pad=0.10)

fig.colorbar(picture, cax=cax, pad = 0.1)
cax.set_ylabel('Covariance / a.u.', fontsize = 14, rotation = 270, labelpad=25)

#directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")
#fig.savefig('TOF_TOF_covariance_indene',bbox_inches="tight")

directory=os.chdir(r"/home/merrickj/Documents/fluorene_ion_figs_and_cov_plots")
fig.savefig('TOF_TOF_covariance_fluorene',bbox_inches="tight")

# directory=os.chdir(r"/home/merrickj/Documents/CPP_ion_figs_and_cov_plots")
# fig.savefig('TOF_TOF_covariance_CPP',bbox_inches="tight")